# Milestone 3 - the daTaBABES

In [15]:
# imports

import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

In [16]:
# Turn off warnings
import warnings

warnings.filterwarnings('ignore')

Users have rated the beers from two websites, RateBeer and BeerAdvocate, thus we get reviews of users from two distinct datasets. The reviews are in txt files, thus we convert the txt files into csv so that we can manipulate the data more easily. By doing this, we get as much as data as we can.

The conversion of the txt files into csv files is done in the notebook 'preprocessing_txt.ipynb'.

Since our goal is to analyze the ratings, what is important here is to see if the ratings from the two websited differ. Can we merge the two datasets without falsing our final results?

In [17]:
# loading the ratings datasets 
ratings_ba_df = pd.read_csv('data/BeerAdvocate/ratings_BA.csv')
ratings_rb_df = pd.read_csv('data/RateBeer/ratings_RB.csv')
ratings_matched_df = pd.read_csv('data/matched_beer_data/ratings.csv')

# loading the users dataset 
users_ba_df = pd.read_csv('data/BeerAdvocate/users.csv')
users_rb_df = pd.read_csv('data/RateBeer/users.csv')
users_matched_df = pd.read_csv('data/matched_beer_data/users.csv')

# loading the beers dataset
beers_ba_df = pd.read_csv('data/BeerAdvocate/beers.csv')
beers_rb_df = pd.read_csv('data/RateBeer/beers.csv')
beers_matched_df = pd.read_csv('data/matched_beer_data/beers.csv')

# Filtering and merging the datasets

## Step 0: Loading the datasets

In [18]:
# Convert the date column to datetime format for BeerAdvocate
ratings_ba_df['date'] = pd.to_datetime(ratings_ba_df['date'], unit='s')
ratings_ba_df['year'] = ratings_ba_df['date'].dt.year
ratings_ba_df['month'] = ratings_ba_df['date'].dt.month

# Convert the date column to datetime format for RateBeer
ratings_rb_df['date'] = pd.to_datetime(ratings_rb_df['date'], unit='s')
ratings_rb_df['year'] = ratings_rb_df['date'].dt.year
ratings_rb_df['month'] = ratings_rb_df['date'].dt.month

In [19]:
# print column name and first row to get the real column name
for col in beers_matched_df.columns:
    print(beers_matched_df[col].head(1))
    print('-----------------------')

0    abv
Name: ba, dtype: object
-----------------------
0    avg
Name: ba.1, dtype: object
-----------------------
0    avg_computed
Name: ba.2, dtype: object
-----------------------
0    avg_matched_valid_ratings
Name: ba.3, dtype: object
-----------------------
0    ba_score
Name: ba.4, dtype: object
-----------------------
0    beer_id
Name: ba.5, dtype: object
-----------------------
0    beer_name
Name: ba.6, dtype: object
-----------------------
0    beer_wout_brewery_name
Name: ba.7, dtype: object
-----------------------
0    brewery_id
Name: ba.8, dtype: object
-----------------------
0    brewery_name
Name: ba.9, dtype: object
-----------------------
0    bros_score
Name: ba.10, dtype: object
-----------------------
0    nbr_matched_valid_ratings
Name: ba.11, dtype: object
-----------------------
0    nbr_ratings
Name: ba.12, dtype: object
-----------------------
0    nbr_reviews
Name: ba.13, dtype: object
-----------------------
0    style
Name: ba.14, dtype: object
--------

## Step 1: Only keep ratings from US users

In [20]:
# extract the country from the location
users_rb_df['location'] = users_rb_df['location'].str.split(',').str[0]
users_ba_df['location'] = users_ba_df['location'].str.split(',').str[0]

In [21]:
US_country = 'United States'

# keep only the rows where the user country is United States
users_ba_df = users_ba_df[users_ba_df.location == US_country]
users_rb_df = users_rb_df[users_rb_df.location == US_country]

# merge the ratings and users data frames to get the location of the user for each rating
ratings_users_ba_df = pd.merge(ratings_ba_df, users_ba_df[['user_id', 'location']], on='user_id', how='inner')
ratings_users_rb_df = pd.merge(ratings_rb_df, users_rb_df[['user_id', 'location']], on='user_id', how='inner')


Filtered dataframe to only keep ratings of users from the US:

In [22]:
# filter the ratings dataframes to only keep the popular ratings of the users from the US
ratings_ba_df = ratings_users_ba_df[ratings_users_ba_df.location == US_country]
ratings_ba_df.drop('location', axis=1, inplace=True)
ratings_rb_df = ratings_users_rb_df[ratings_users_rb_df.location == US_country]
ratings_rb_df.drop('location', axis=1, inplace=True)

### Are there users who excessively rate a beer in a short time period ?

Motivation: we want to handle the cases where we could have many ratings from a specific user for the same beer as it could be a user that is trying to bias the rating of a particular beer

In [23]:
# create a year_month column
ratings_ba_df['year_month'] = ratings_ba_df['year'].astype(str) + '-' + ratings_ba_df['month'].astype(str)
ratings_rb_df['year_month'] = ratings_rb_df['year'].astype(str) + '-' + ratings_rb_df['month'].astype(str)

# convert the year_month column to datetime format
ratings_ba_df['year_month'] = pd.to_datetime(ratings_ba_df['year_month'])
ratings_rb_df['year_month'] = pd.to_datetime(ratings_rb_df['year_month'])


In [24]:
# Assuming ratings_combined_df is your DataFrame
threshold = 3  # Adjust this threshold based on your criteria

# Group by user_id, beer_id, and year_month, and count the number of ratings
user_beer_counts_ba = ratings_ba_df.groupby(['user_id', 'beer_id', 'year_month']).size().reset_index(name='rating_count')
user_beer_counts_rb = ratings_rb_df.groupby(['user_id', 'beer_id', 'year_month']).size().reset_index(name='rating_count')

# Filter for instances where the rating count exceeds the threshold
excessive_ratings_ba = user_beer_counts_ba[user_beer_counts_ba['rating_count'] > threshold]
excessive_ratings_rb = user_beer_counts_rb[user_beer_counts_rb['rating_count'] > threshold]

print("Excessive ratings for BeerAdvocate:")
print(excessive_ratings_ba)
print("Excessive ratings for RateBeer:")
print(excessive_ratings_rb)

Excessive ratings for BeerAdvocate:
Empty DataFrame
Columns: [user_id, beer_id, year_month, rating_count]
Index: []
Excessive ratings for RateBeer:
         user_id  beer_id year_month  rating_count
184060      3070      252 2002-11-01             5
184061      3070      403 2002-01-01             5
184062      3070      411 2002-01-01             5
184063      3070      474 2002-01-01             5
184064      3070      707 2002-01-01             5
1474422    46797       46 2007-01-01            41
1474423    46797       86 2007-03-01            41
1474424    46797       87 2007-01-01            41
1474425    46797      145 2007-01-01            41
1474426    46797      589 2007-02-01            41
1474427    46797      594 2007-02-01            41
1474428    46797      662 2007-03-01            41
1474429    46797      663 2007-03-01            41
1474430    46797      690 2007-01-01            41
1474431    46797      714 2007-01-01            41
1474432    46797      745 2007-02-01

We decided to set the threshold to 3 as rating a same beer more than 3 times per month can seem suspicious. We have identified two instances where users have rated the same beer excessively within a specific month. For example, user 46797 has rated 41 times a many beers each month from december 2006 to march 2007.  
Such behavior is suspicious, and as a precautionary measure, we've decided to remove these users from the database to keep the reliability of the ratings.

In [25]:
# Remove the suspicious users
users_to_remove = [46797, 3070]
ratings_combined_df = ratings_ba_df[~ratings_ba_df['user_id'].isin(users_to_remove)]


## Step 2: Only keep good ratings

In [26]:
mean_ba = ratings_ba_df.rating.mean()
mean_rb = ratings_rb_df.rating.mean()

print("Mean rating in BeerAdvocate:", mean_ba)
print("Mean rating in RateBeer:", mean_rb)

Mean rating in BeerAdvocate: 3.846906563579535
Mean rating in RateBeer: 3.3793541115704993


In [27]:
top_ratings_by_style_ba = ratings_ba_df.groupby('style')['rating'].mean().sort_values(ascending=False)
top_ratings_by_style_rb = ratings_rb_df.groupby('style')['rating'].mean().sort_values(ascending=False)

print(f"Number of Beer Styles in AdvocateBeer: {len(top_ratings_by_style_ba)}")
print(f"Number of Beer Styles in RateBeer: {len(top_ratings_by_style_rb)}")

Number of Beer Styles in AdvocateBeer: 104
Number of Beer Styles in RateBeer: 90


Filtered dataframes to only keep the rating bigger than mean average rating:

In [28]:
ratings_ba_df_untouched = ratings_ba_df.copy()
ratings_rb_df_untouched = ratings_rb_df.copy()
ratings_matched_df_untouched = ratings_matched_df.copy()

# Filter the ratings dataframes to only keep the popular ratings
ratings_ba_df = ratings_ba_df[ratings_ba_df['rating'] > mean_ba]
ratings_rb_df = ratings_rb_df[ratings_rb_df['rating'] > mean_rb]


## Step 3: Names have to match between the two datasets

In [ ]:
# New df without beers in beers_ba_df that are not in ratings_ba_df
beers_ba_df_considered = beers_ba_df[beers_ba_df['beer_id'].isin(ratings_ba_df['beer_id'])]
# Remove beers in beers_rb_df that are not in ratings_rb_df
beers_rb_df_considered = beers_rb_df[beers_rb_df['beer_id'].isin(ratings_rb_df['beer_id'])]

Motivation: We noticed that beer styles names vary across the two datasets and we want them to match to be able to process the data correctly.

In the matched beer dataset, we only have a limited subset of the entire datasets. Consequently, if we aim to replace the style names of beers in the matched dataset using beer_id, we will only have matching names for a small fraction of all the beers in the datasets. Therefore, we will base our conversion on the beer style.

In [29]:
beer_ids_ba_not_matched = beers_ba_df['beer_id'][~beers_ba_df['beer_id'].isin(beers_matched_df['ba.5'])]
print(f'Percentage of beers in BeerAdvocate not in the matched dataset: {len(beer_ids_ba_not_matched) / beers_ba_df.shape[0]}')

beer_ids_rb_not_matched = beers_rb_df['beer_id'][~beers_rb_df['beer_id'].isin(beers_matched_df['rb.4'])]
print(f'Percentage of beers in RateBeer not in the matched dataset: {len(beer_ids_rb_not_matched) / beers_rb_df.shape[0]}')

Percentage of beers in BeerAdvocate not in the matched dataset: 0.8958169380713118
Percentage of beers in RateBeer not in the matched dataset: 0.9338198203496644


In [30]:
# Number of styles in BA and RB in matched dataset
total_groups = len(beers_matched_df[1:].groupby('ba.14'))
print(f'Total number of styles in BA in matched dataset: {total_groups}')

total_groups = len(beers_matched_df[1:].groupby('rb.12'))
print(f'Total number of styles in RB in matched dataset: {total_groups}')

Total number of styles in BA in matched dataset: 104
Total number of styles in RB in matched dataset: 80


All styles in BeerAdvocate appear in the matched dataset, and only 14 in RateBeer don't:

In [31]:
styles_ba_not_matched = beers_ba_df['style'][~beers_ba_df['style'].isin(beers_matched_df['ba.14'])]
unique_styles_ba_not_matched = styles_ba_not_matched.value_counts()
print(f'Number of styles in BA not in matched: {len(unique_styles_ba_not_matched)}')
for style, count in unique_styles_ba_not_matched.items():
    print(f"{style} --> {count} occurrences")

styles_rb_not_matched = beers_rb_df['style'][~beers_rb_df['style'].isin(beers_matched_df['rb.12'])]
unique_styles_rb_not_matched = styles_rb_not_matched.value_counts()
print(f'Number of styles in RB not in matched: {len(unique_styles_rb_not_matched)}')
for style, count in unique_styles_rb_not_matched.items():
    print(f"{style} --> {count} occurrences")

Number of styles in BA not in matched: 0
Number of styles in RB not in matched: 14
Perry --> 176 occurrences
Saké - Junmai --> 24 occurrences
Saké - Infused --> 14 occurrences
Saké - Daiginjo --> 13 occurrences
Saké - Ginjo --> 11 occurrences
Saké - Nigori --> 9 occurrences
Saké - Futsu-shu --> 7 occurrences
Saké - Namasaké --> 6 occurrences
Saké - Tokubetsu --> 4 occurrences
Saké - Koshu --> 3 occurrences
Saké - Honjozo --> 3 occurrences
Saké - Taru --> 2 occurrences
Saké - Genshu --> 2 occurrences
Ice Cider/Ice Perry --> 2 occurrences


Since there are more styles in BA than in RB, we will convert the name styles in RB to match the corresponding name in BA that is the most used:

In [32]:
# For each style in BA, show the most used corresponding style name in RB

for style1, group_style2 in beers_matched_df[1:].groupby('ba.14'):
    styles2_counts = group_style2['rb.12'].value_counts().head(5)
    print(f'{style1} --> {styles2_counts}')
    print('-----------------------')


Altbier --> rb.12
Altbier              203
Amber Ale              7
Traditional Ale        4
Sour/Wild Ale          2
American Pale Ale      1
Name: count, dtype: int64
-----------------------
American Adjunct Lager --> rb.12
Pale Lager            78
Premium Lager         17
Pilsener              11
Specialty Grain        4
Amber Lager/Vienna     4
Name: count, dtype: int64
-----------------------
American Amber / Red Ale --> rb.12
Amber Ale               932
American Strong Ale     117
India Pale Ale (IPA)     60
Irish Ale                48
Spice/Herb/Vegetable     27
Name: count, dtype: int64
-----------------------
American Amber / Red Lager --> rb.12
Amber Lager/Vienna    61
Amber Ale             14
Premium Lager         10
Pale Lager             4
Dunkel/Tmavý           3
Name: count, dtype: int64
-----------------------
American Barleywine --> rb.12
Barley Wine             291
American Strong Ale       3
Spice/Herb/Vegetable      2
Imperial IPA              2
Abt/Quadrupel       

In [33]:
# For each style in RB, show the most used corresponding style name in BA
style_rb_to_ba_dict = {}

for style2, group_style1 in beers_matched_df[1:].groupby('rb.12'):
    styles1_counts = group_style1['ba.14'].value_counts().head(5)
    print(f'{style2} --> {styles1_counts}')
    style_rb_to_ba_dict[style2] = styles1_counts.index[0]
    print('-----------------------')

Abbey Dubbel --> ba.14
Dubbel                     288
Belgian Strong Dark Ale     20
Belgian Dark Ale            17
American Pale Ale (APA)      4
Saison / Farmhouse Ale       2
Name: count, dtype: int64
-----------------------
Abbey Tripel --> ba.14
Tripel                     430
Belgian Strong Pale Ale     18
Belgian IPA                  6
Belgian Pale Ale             2
American Wild Ale            1
Name: count, dtype: int64
-----------------------
Abt/Quadrupel --> ba.14
Quadrupel (Quad)           165
Belgian Strong Dark Ale     15
Belgian Strong Pale Ale      2
American Barleywine          1
Dubbel                       1
Name: count, dtype: int64
-----------------------
Altbier --> ba.14
Altbier                     203
American Amber / Red Ale      7
Kölsch                        1
Munich Helles Lager           1
Dunkelweizen                  1
Name: count, dtype: int64
-----------------------
Amber Ale --> ba.14
American Amber / Red Ale      932
Irish Red Ale                  21

We obtain the following beer style conversion dictionnary:

In [34]:
style_rb_to_ba_dict

{'Abbey Dubbel': 'Dubbel',
 'Abbey Tripel': 'Tripel',
 'Abt/Quadrupel': 'Quadrupel (Quad)',
 'Altbier': 'Altbier',
 'Amber Ale': 'American Amber / Red Ale',
 'Amber Lager/Vienna': 'Vienna Lager',
 'American Pale Ale': 'American Pale Ale (APA)',
 'American Strong Ale ': 'American Strong Ale',
 'Baltic Porter': 'Baltic Porter',
 'Barley Wine': 'American Barleywine',
 'Belgian Ale': 'Belgian Pale Ale',
 'Belgian Strong Ale': 'Belgian Strong Pale Ale',
 'Berliner Weisse': 'Berliner Weissbier',
 'Bitter': 'English Bitter',
 'Bière de Garde': 'Bière de Garde',
 'Black IPA': 'American Black Ale',
 'Brown Ale': 'American Brown Ale',
 'California Common': 'California Common / Steam Beer',
 'Cider': 'Fruit / Vegetable Beer',
 'Cream Ale': 'Cream Ale',
 'Czech Pilsner (Světlý)': 'Czech Pilsener',
 'Doppelbock': 'Doppelbock',
 'Dortmunder/Helles': 'Munich Helles Lager',
 'Dry Stout': 'Irish Dry Stout',
 'Dunkel/Tmavý': 'Munich Dunkel Lager',
 'Dunkelweizen': 'Dunkelweizen',
 'Dunkler Bock': 'Bock'

We have almost 3 times more ratings in Ratebeer than in BeerAdvocate, and some conversion in the dictionnary don't make sense since we are converting a broader style name present in RateBeer into a more specific style name present in BeerAdvocate. For example while in Ratebeer we only have the "Barley Wine" style, we have "American Barleywine" and "English Barleywine" style in BeerAdvocate and we can't convert it into either one of them without having more information on the beer (but again we only have a very small portion of the beers in the matched dataset so we can't use that). However since we have 3 times more beer in RateBeer than in BeerAdvocate, we decided that we will convert for those specific case the more specific names in BeerAdvocate into the broader style name in RateBeer.

In [35]:
nbr_ratings_ba = ratings_ba_df.shape[0]
nbr_ratings_rb = ratings_rb_df.shape[0]

print('Number of ratings in BeerAdvocate: {}'.format(nbr_ratings_ba))
print('Number of ratings in RateBeer: {}'.format(nbr_ratings_rb))

Number of ratings in BeerAdvocate: 1358021
Number of ratings in RateBeer: 1863770


Convert the name of beer styles in the RateBeer beers and ratings datasets:

In [36]:
ratings_rb_df['style'] = ratings_rb_df['style'].replace(style_rb_to_ba_dict)
beers_rb_df['style'] = beers_rb_df['style'].replace(style_rb_to_ba_dict)

### Step 4: Merge the rating datasets from BeerAdvocate and RateBeer

In [37]:
# merge the two ratings data frames
ratings_combined_df_untouched = pd.concat([ratings_ba_df_untouched, ratings_rb_df_untouched], ignore_index=True)
ratings_combined_df = pd.concat([ratings_ba_df, ratings_rb_df], ignore_index=True)


In [38]:
# group by year_month and count the number of ratings per year_month
year_month_ratings_count_ba = ratings_ba_df.groupby('year_month')['year_month'].count()
year_month_ratings_count_rb = ratings_rb_df.groupby('year_month')['year_month'].count()

print('Earliest date of ratings:', ratings_ba_df['date'].min())
print('Latest date of ratings:', ratings_ba_df['date'].max())

Earliest date of ratings: 1996-08-22 10:00:00
Latest date of ratings: 2017-08-01 10:00:00
